In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision

from torchvision import transforms
from sklearn.metrics import average_precision_score
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from kaggle_submission import output_submission_csv
from classifier import SimpleClassifier, Classifier#, AlexNet
from voc_dataloader import VocDataset, VOC_CLASSES

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
x = torch.rand(5,3)
x


tensor([[0.0898, 0.7925, 0.7235],
        [0.9800, 0.2232, 0.5986],
        [0.5435, 0.0565, 0.3281],
        [0.0263, 0.1573, 0.0986],
        [0.2740, 0.2347, 0.7344]])

In [4]:
torch.tensor([1.3,2,3]).dtype

torch.float32

In [5]:
torch.zeros(4,5,6)

tensor([[[0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.]]])

In [6]:
x.new_ones(5,8)

tensor([[1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [7]:
x = x.to(dtype=torch.int8)
x

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]], dtype=torch.int8)

In [8]:
x = torch.rand(4,7)
y = torch.rand(4,7)
x+y
torch.add(x,y).to(dtype=torch.int8)

tensor([[0, 0, 1, 0, 0, 1, 1],
        [1, 1, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0, 0],
        [1, 1, 1, 1, 0, 1, 1]], dtype=torch.int8)

In [9]:
y.add(x)

tensor([[0.4821, 0.4851, 1.5130, 0.3849, 0.6817, 1.0579, 1.7308],
        [1.0018, 1.6768, 0.9388, 0.7442, 0.6886, 1.4887, 0.8832],
        [0.9528, 0.7525, 0.4577, 1.4988, 0.5220, 0.7319, 0.7235],
        [1.2408, 1.0052, 1.0551, 1.2943, 0.3265, 1.1430, 1.3055]])

In [10]:
y

tensor([[0.0646, 0.3498, 0.9034, 0.1877, 0.3482, 0.2686, 0.7418],
        [0.0556, 0.7576, 0.2908, 0.3551, 0.4086, 0.5010, 0.2246],
        [0.1010, 0.5699, 0.2012, 0.7336, 0.0799, 0.0398, 0.0084],
        [0.5636, 0.3671, 0.0736, 0.7285, 0.0273, 0.6426, 0.4693]])

In [11]:
y.add_(x)

tensor([[0.4821, 0.4851, 1.5130, 0.3849, 0.6817, 1.0579, 1.7308],
        [1.0018, 1.6768, 0.9388, 0.7442, 0.6886, 1.4887, 0.8832],
        [0.9528, 0.7525, 0.4577, 1.4988, 0.5220, 0.7319, 0.7235],
        [1.2408, 1.0052, 1.0551, 1.2943, 0.3265, 1.1430, 1.3055]])

In [12]:
y

tensor([[0.4821, 0.4851, 1.5130, 0.3849, 0.6817, 1.0579, 1.7308],
        [1.0018, 1.6768, 0.9388, 0.7442, 0.6886, 1.4887, 0.8832],
        [0.9528, 0.7525, 0.4577, 1.4988, 0.5220, 0.7319, 0.7235],
        [1.2408, 1.0052, 1.0551, 1.2943, 0.3265, 1.1430, 1.3055]])

In [13]:
-y

tensor([[-0.4821, -0.4851, -1.5130, -0.3849, -0.6817, -1.0579, -1.7308],
        [-1.0018, -1.6768, -0.9388, -0.7442, -0.6886, -1.4887, -0.8832],
        [-0.9528, -0.7525, -0.4577, -1.4988, -0.5220, -0.7319, -0.7235],
        [-1.2408, -1.0052, -1.0551, -1.2943, -0.3265, -1.1430, -1.3055]])

In [14]:
x = torch.rand(5,2)
y = torch.rand(5,1)

x + y

tensor([[0.9337, 1.4603],
        [1.3265, 1.4912],
        [1.0971, 0.2709],
        [0.5387, 0.3681],
        [1.4073, 1.0540]])

In [15]:
x

tensor([[0.3092, 0.8358],
        [0.4384, 0.6032],
        [0.9913, 0.1650],
        [0.2814, 0.1108],
        [0.6264, 0.2731]])

In [16]:
y

tensor([[0.6245],
        [0.8881],
        [0.1059],
        [0.2572],
        [0.7809]])

In [17]:
x = torch.randn(4,4)

In [18]:
x.view(16)

tensor([-1.5237, -0.4292, -0.3865,  0.1526,  2.3103,  0.3731, -0.3461,  1.4719,
        -0.1791,  1.0314, -0.0318,  1.9965,  2.1557,  0.0990,  0.2684,  0.2361])

In [19]:
x.view(-1,8)

tensor([[-1.5237, -0.4292, -0.3865,  0.1526,  2.3103,  0.3731, -0.3461,  1.4719],
        [-0.1791,  1.0314, -0.0318,  1.9965,  2.1557,  0.0990,  0.2684,  0.2361]])

In [20]:
x.view(2,8)

tensor([[-1.5237, -0.4292, -0.3865,  0.1526,  2.3103,  0.3731, -0.3461,  1.4719],
        [-0.1791,  1.0314, -0.0318,  1.9965,  2.1557,  0.0990,  0.2684,  0.2361]])

In [41]:
conda install -c anaconda cudatoolkit

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


x.reshape(2,8)

In [22]:
if torch.cuda.is_available():
    !nvidia-smi

Wed Mar 30 22:19:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 471.41       Driver Version: 471.41       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   54C    P8    14W /  N/A |   4980MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [23]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    y = torch.ones_like(x,device = device)
    x = x.to(device)
    z = x + y
    print(z)
    print(z.to("cpu",torch.double))

tensor([[-0.5237,  0.5708,  0.6135,  1.1526],
        [ 3.3103,  1.3731,  0.6539,  2.4719],
        [ 0.8209,  2.0314,  0.9682,  2.9965],
        [ 3.1557,  1.0990,  1.2684,  1.2361]], device='cuda:0')
tensor([[-0.5237,  0.5708,  0.6135,  1.1526],
        [ 3.3103,  1.3731,  0.6539,  2.4719],
        [ 0.8209,  2.0314,  0.9682,  2.9965],
        [ 3.1557,  1.0990,  1.2684,  1.2361]], dtype=torch.float64)


In [26]:
# automated differnetiation
x = torch.ones(2,2,requires_grad=True)
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [35]:
y = x*(x + 2 + x *7)
z = y + y
z

tensor([[20., 20.],
        [20., 20.]], grad_fn=<AddBackward0>)

In [31]:
a = torch.rand(5,6)
a.requires_grad

False

In [39]:
print(y.grad)

None


In [47]:
x = torch.rand(4,7,requires_grad=True)
y = x * 2
while y.data.norm() < 1000:
    y = y *2
print(y)

tensor([[ 99.5037, 332.5784, 384.9811,   9.4921, 246.0810, 121.6165, 225.7818],
        [  5.0806,  22.7139, 454.3484, 284.6530, 260.5345,  57.7639, 109.0172],
        [ 12.6169,  60.3246, 478.0029, 478.8306, 453.1118, 165.2356, 177.9507],
        [400.6879, 403.0247, 399.9465, 304.6674, 454.6248, 234.0269,  76.7852]],
       grad_fn=<MulBackward0>)


In [48]:
print(y.grad)

None


C:\Users\Nbsyxx\anaconda3\lib\site-packages\torch\_tensor.py:1104: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:475.)
  return self._grad


In [49]:
gradient = torch.rand(4,7)

In [51]:
print(x.grad)

None


In [52]:
y.backward(gradient)

In [53]:
x.grad

tensor([[154.1935, 156.7552, 116.2405, 296.5452, 251.5618, 335.6506, 382.8904],
        [171.6852,  39.9842, 113.3427, 209.6156, 174.8719, 297.6234, 336.6946],
        [ 21.8764, 352.8730, 177.1976,  68.7212, 106.8423, 106.8274, 115.4985],
        [342.7648,  81.9271, 137.6261, 213.2461,  52.4958, 108.7299, 275.6191]])